# Chapter 11 - Dropout与学习率衰减

在机器学习中存在两类比较严重的问题: 过拟合和学习时间开消费非常大。为了解决过拟合问题通常会采用组合方法，即训练多个模型来解决问题，这样会带来非常大的时间开销，Dropout可以很好地解决这个问题。另一个时间开销巨大的地方是梯度下降，学习率衰减可以很好地解决梯度下降中的时间开销问题。

## 11.1 神经网络中的Dropout

Dropout是Srivastava等人在2014年的一篇论文([Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf?utm_content=buffer79b43&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer))中提出的一种针对神经网络模型的正则化方法。Dropout是在训练过程中随机忽略部分的神经元。也就是说，在正向传播过程中，这些被忽略的神经元对下游神经元的贡献效果暂时消失；在反向传播时，这些神经元也不会有任何权值的更新。

随着神经网络模型不断地学习，神经元的权值会与整个网络的上下文匹配。神经元的权重针对某些特征进行调优，使其特殊化，周围的神经元则会依赖于这种特殊化。如果过于特殊化，模型就会因为队训练数据过拟合而脆弱不堪。如果在训练过程中随即丢弃网络的一部分，那么其他神经元就不得不介入，替代缺失那部分神经元的内部表征。这么做的效果就是，网络模型对神经元特定的权重就不那么敏感。因此提升了模型的泛化能力，不容易对训练过拟合。

从直观上来讲，Dropout是ensemble在分类性能上的一个近似。然而在实际中，Dropout毕竟还是在一个神经网络上进行的，只训练出一套模型参数。那么它到底因何而有效呢？论文中作者对dropout的动机做了一个十分精彩的类比：

在自然界中，在中大型动物中，一般是有性繁殖，有性繁殖是指后代的基因从父母两方各继承一半。但是从直观上看，似乎无性繁殖更加合理，因为无性繁殖可以保留大段大段的优秀基因。而有性繁殖则将基因随机拆了又拆，破坏了大段基因的联合适应性。但是自然选择中毕竟没有选择无性繁殖，而选择了有性繁殖，须知物竞天择，适者生存。我们先做一个假设，那就是基因的力量在于混合的能力而非单个基因的能力。不管是有性繁殖还是无性繁殖都得遵循这个假设。为了证明有性繁殖的强大，我们先看一个概率学小知识。

比如要搞一次恐怖袭击，两种方式： 
- 集中50人，让这50个人密切精准分工，搞一次大爆破。 
- 将50人分成10组，每组5人，分头行事，去随便什么地方搞点动作，成功一次就算。

哪一个成功的概率比较大？ 显然是后者。因为将一个大团队作战变成了游击战。

那么，类比过来，有性繁殖的方式不仅仅可以将优秀的基因传下来，还可以降低基因之间的联合适应性，使得复杂的大段大段基因联合适应性变成比较小的一个一个小段基因的联合适应性。

dropout也能达到同样的效果，它强迫一个神经单元，和随机挑选出来的其他神经单元共同工作，达到好的效果。消除减弱了神经元节点间的联合适应性，增强了泛化能力。

如何选择Dropout率呢？经过验证，隐含节点Dropout率等于0.5的时候效果最好，此时Dropout随机生成的网络结构最多。Dropout层也可以用在输入层，作为一种添加噪声的方法。输入层设为更接近1的数字，是的输入变化不会太大(如0.8)。

[参考网页](https://blog.csdn.net/stdcoutzyx/article/details/49022443)

## 11.2 在Keras里使用Dropout 

In [2]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [3]:
# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# 设定随机种子
seed = 7
np.random.seed(seed)

In [4]:
# 基准模型

# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #定义Dropout
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [5]:
model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Accuracy: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy: 80.67% (0.21)


### 11.2.1 输入层使用Dropout
在输入层后添加一个新的Dropout层，Dropout率设置为20%，意味着每个更新周期中20%的输入都将被随即排除。

In [6]:
# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dropout(rate=0.2, input_shape=(4,)))
    model.add(Dense(units=4, activation='relu', kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    # 定义Dropout
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [7]:
model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Accuracy: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy: 82.00% (0.11)


### 11.2.2 在隐藏层中使用Dropout
在两个隐藏层之间，以及最后一个隐藏层和输出层之间使用Dropout，也将Dropout率设置为20%，并对权重进行约束，使其最大限度不超过3。

In [10]:
from keras.constraints import maxnorm

In [11]:
# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init, kernel_constraint=maxnorm(3)))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init, kernel_constraint=maxnorm(3)))
    model.add(Dropout(rate=0.2))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    # 定义Dropout
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [12]:
model = KerasClassifier(build_fn=create_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Accuracy: %.2f%% (%.2f)' % (results.mean()*100, results.std()))

Accuracy: 78.67% (0.22)


### 11.2.3 Dropout使用技巧

在Dropout: A Simple Way to Prevent Neural Networks from Overfitting这篇论文中，提供了一套标准机器学习问题的实验结果。在这些实验的实践中提供了一些Dropout的使用建议：
* 通常在神经网络中使用20%~50%的Dropout率，20%是一个很好的起点。太低的概率产生的作用有限；太高的概率可能导致对网络的训练不充分。
* 当在较大的网络上使用Dropout时，可能会获得更好的表现，因为Dropout降低了模型训练过程中的干扰。
* 在输入层(可视层)和隐藏层上使用Dropout。在网络的每一层上使用Dropout都显示出良好的效果。
* 使用较高的学习率，并使用学习率衰减和巨大的动量值，将学习率提高10~100倍，且使用0.9或0.99的高动量值。
* 限制网络权重的大小。大的学习率可能导致非常大的网络权重。对网络权重的大小施加约束，例如大小为4或5的最大范数正则化(Max-norm Regularizition)，已经显示出具有很好的改善效果。在Keras中，可以通过指定Dense的kernel_constrain=maxnorm(x)来限制网络权重。

## 11.3 学习率衰减

### 11.3.1 学习率线性衰减

In [ ]:
# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #模型优化
    learningRate = 0.1
    momentum = 0.9
    decay_rate = 0.005
    sgd = SGD(lr=learningRate, momentum=momentum, decay=decay_rate, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [ ]:
epochs = 200
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=5, verbose=1)
model.fit(x, Y)

### 11.3.2 学习率指数衰减

In [ ]:
# 计算学习率
def step_decay(epoch):
    init_lrate = 0.1
    drop = 0.5
    epochs_drop = 10
    lrate = init_lrate * pow(drop, floor(1 + epoch) / epochs_drop)
    return lrate

In [ ]:
# 构建模型函数
def create_model(init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    #模型优化
    learningRate = 0.1
    momentum = 0.9
    decay_rate = 0.0
    sgd = SGD(lr=learningRate, momentum=momentum, decay=decay_rate, nesterov=False)

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    return model

In [ ]:
lrate = LearningRateScheduler(step_decay)
epochs = 200
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=5, verbose=1, callbacks=[lrate])
model.fit(x, Y)

### 11.3.3 学习率衰减的使用技巧

在深度学习中使用学习率衰减，通常会考虑以下情况：
* **提高初始学习率。**更大的学习率，在开始学习时会快速更新权重值，而且随着学习率的衰减可以自动调整学习率，这可以提高梯度下降的性能。
* **使用大动量。**使用较大的动量值将有助于优化算法在学习率缩小到小值时，继续向正确方向更新权重。